In [ ]:
# IMPORT PACKAGES
import os
import cv2
import math
import random
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from IPython.display import clear_output
from random import randint
from glob import glob
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, LabelEncoder
from skmultilearn.model_selection import iterative_train_test_split
import albumentations as A

# TensorFlow / Keras Imports
from tensorflow.keras import backend as K
from tensorflow.keras import layers, Model, Sequential
from tensorflow.keras.applications import ResNet101  
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
)
from tensorflow.keras.metrics import MeanIoU, Recall, Precision
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.utils import Sequence
from tensorflow.keras.mixed_precision import set_global_policy, Policy

# Keras Layers
from tensorflow.keras.layers import (
    Input, Dense, Conv2D, Conv2DTranspose, BatchNormalization, Activation,
    ReLU, LeakyReLU, Add, Softmax, Dropout, UpSampling2D, concatenate, 
    ZeroPadding2D, Lambda, Reshape, GlobalAveragePooling2D, 
    AveragePooling2D, MaxPooling2D, multiply, add
)

clear_output()

def convolution_block(block_input, num_filters=256, kernel_size=3, dilation_rate=1, padding='same', use_bias=False):
    x = Conv2D(num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, 
               padding=padding, use_bias=use_bias, kernel_initializer=he_normal())(block_input)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    return Activation('relu')(x)

# Dilated Spatial Pyramid Pooling with adjusted dilation rates
def DilatedSpatialPyramidPooling(dspp_input):
    dims = list(dspp_input.shape)
    x = AveragePooling2D(pool_size=(dims[1], dims[2]))(dspp_input)
    x = convolution_block(x, num_filters=256, kernel_size=1, use_bias=True)
    out_pool = UpSampling2D(size=(dims[1] // x.shape[1], dims[2] // x.shape[2]), interpolation='bilinear')(x)

    out_1 = convolution_block(dspp_input, num_filters=256, kernel_size=1, dilation_rate=1)
    out_3 = convolution_block(dspp_input, num_filters=256, kernel_size=3, dilation_rate=3)  # Added smaller dilation rate
    out_9 = convolution_block(dspp_input, num_filters=256, kernel_size=3, dilation_rate=9)
    out_15 = convolution_block(dspp_input, num_filters=256, kernel_size=3, dilation_rate=15)
    out_21 = convolution_block(dspp_input, num_filters=256, kernel_size=3, dilation_rate=21)  # Added larger dilation rate

    # Apply Dropout layers to out_3, out_9, and out_15
    out_3 = Dropout(0.2)(out_3)
    out_9 = Dropout(0.2)(out_9)
    out_15 = Dropout(0.2)(out_15)

    x = concatenate([out_pool, out_1, out_3, out_9, out_15, out_21], axis=-1)  # Combine multi-scale features
    output = convolution_block(x, num_filters=256, kernel_size=1)
    return output

# Full Model without Morphological Post-Processing Layer
def DeeplabV3Plus(image_size, num_classes):
    input = Input(shape=(image_size, image_size, 3))
    resnet101 = ResNet101(weights="imagenet", include_top=False, input_tensor=input)

    x = resnet101.get_layer("conv4_block23_out").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]), interpolation="bilinear"
    )(x)
    input_b = resnet101.get_layer("conv2_block3_out").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    input_b = UpSampling2D(
        size=(input_a.shape[1] // input_b.shape[1], input_a.shape[2] // input_b.shape[2]),
        interpolation="bilinear",
    )(input_b)
    x = concatenate([input_a, input_b], axis=-1)

    x = convolution_block(x, num_filters=256)
    x = convolution_block(x, num_filters=256)
    x = convolution_block(x, num_filters=128)

    x = UpSampling2D(size=(image_size // x.shape[1], image_size // x.shape[2]), interpolation="bilinear")(x)
    output = Conv2D(num_classes, kernel_size=(1, 1), padding="same", activation="softmax")(x)

    return Model(inputs=input, outputs=output, name="DeepLabV3")

